In [0]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window




In [0]:
silver_erp_CUST_AZ12_df=spark.read.format("delta").load("/Volumes/etl/medal/bronze/erp/erp_CUST_AZ12")
silver_erp_PX_CAT_G1V2_df=spark.read.format("delta").load("/Volumes/etl/medal/bronze/erp/erp_PX_CAT_G1V2")
silver_erp_LOC_A101_df=spark.read.format("delta").load("/Volumes/etl/medal/bronze/erp/erp_LOC_A101")
silver_crm_cust_df=spark.read.format("delta").load("/Volumes/etl/medal/bronze/crm/crm_cust")
silver_crm_prd_info_df=spark.read.format("delta").load("/Volumes/etl/medal/bronze/crm/crm_prd_info")
silver_crm_sales_details_df=spark.read.format("delta").load("/Volumes/etl/medal/bronze/crm/crm_sales_details")



In [0]:
silver_crm_cust_df = silver_crm_cust_df.select(
    "cst_id","cst_key",
    F.trim(F.col("cst_firstname")).alias("cst_firstname"),
     F.trim(F.col("cst_lastname")).alias("cst_lastname"),
    F.when(F.upper(F.trim(F.col("cst_marital_status"))) == "S", "SINGLE")
     .when(F.upper(F.trim(F.col("cst_marital_status"))) == "M", "MARRIED")
     .otherwise("N/A")
     .alias("cst_marital_status"),
    F.when(F.upper(F.trim(F.col("cst_gndr"))) == "M", "MALE")
     .when(F.upper(F.trim(F.col("cst_gndr"))) == "F", "FEMALE")
     .otherwise("N/A")
     .alias("cst_gndr"),
    "cst_create_date"
)

display(silver_crm_cust_df.limit(10))


cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date
11000,AW00011000,Jon,Yang,MARRIED,MALE,2025-10-06
11001,AW00011001,Eugene,Huang,SINGLE,MALE,2025-10-06
11002,AW00011002,Ruben,Torres,MARRIED,MALE,2025-10-06
11003,AW00011003,Christy,Zhu,SINGLE,FEMALE,2025-10-06
11004,AW00011004,Elizabeth,Johnson,SINGLE,FEMALE,2025-10-06
11005,AW00011005,Julio,Ruiz,SINGLE,MALE,2025-10-06
11006,AW00011006,Janet,Alvarez,SINGLE,FEMALE,2025-10-06
11007,AW00011007,Marco,Mehta,MARRIED,MALE,2025-10-06
11008,AW00011008,Rob,Verhoff,SINGLE,FEMALE,2025-10-06
11009,AW00011009,Shannon,Carlson,SINGLE,MALE,2025-10-06


In [0]:
w = Window.partitionBy("cst_id").orderBy(F.col("cst_create_date").desc())

silver_crm_cust_df = silver_crm_cust_df \
    .withColumn("row_num", F.row_number().over(w)) \
    .filter((F.col("row_num") == 1) & (F.col("cst_id").isNotNull())) \
    .drop("row_num")

silver_crm_cust_df.write.mode("overwrite").format("delta")\
.save("/Volumes/etl/medal/silver/crm//crm_cust")

In [0]:
window=Window.partitionBy("prd_id").orderBy(F.col("prd_end_dt"))
silver_crm_prd_info_df.withColumn("row_num", F.row_number().over(window)).filter(F.col("row_num") > 1).display()

prd_id,prd_key,prd_nm,prd_cost,prd_line,prd_start_dt,prd_end_dt,row_num


In [0]:
silver_crm_prd_info_df.select("prd_line").distinct().show()

+--------+
|prd_line|
+--------+
|    NULL|
|      T |
|      S |
|      M |
|      R |
+--------+



In [0]:
window = Window.partitionBy("prd_key").orderBy(F.col("prd_start_dt"))

silver_crm_prd_info_df = silver_crm_prd_info_df.select(
    "prd_id",
    F.regexp_replace(F.substring(F.col("prd_key"), 1, 5), "-", "_").alias("category_id"),
    
    F.substring(F.col("prd_key"), 7, F.length(F.col("prd_key"))).alias("prd_key"),
    "prd_nm",
    F.when(F.col("prd_cost").isNull(), 0)
     .otherwise(F.abs(F.col("prd_cost")))
     .alias("prd_cost"),
    F.when(F.col("prd_line").isNull(), "N/A")
     .when(F.trim(F.col("prd_line")) == "M", "Mountain")
     .when(F.trim(F.col("prd_line")) == "R", "Road")
     .when(F.trim(F.col("prd_line")) == "S", "Other Sales")
     .when(F.trim(F.col("prd_line")) == "T", "Touring")
     .otherwise(F.trim(F.col("prd_line")))
     .alias("prd_line"),
    F.to_date("prd_start_dt", "yyyy-MM-dd").alias("prd_start_dt"),
    (F.lead("prd_start_dt").over(window) - F.expr("INTERVAL 1 DAY")).alias("end_date")
)


silver_crm_prd_info_df.show(10)


silver_crm_prd_info_df.write.mode("overwrite").format("delta") \
    .save("/Volumes/etl/medal/silver/crm/crm_prd_info")

+------+-----------+-------+--------------------+--------+-----------+------------+----------+
|prd_id|category_id|prd_key|              prd_nm|prd_cost|   prd_line|prd_start_dt|  end_date|
+------+-----------+-------+--------------------+--------+-----------+------------+----------+
|   478|      AC_BC|BC-M005|Mountain Bottle Cage|       4|   Mountain|  2013-07-01|      NULL|
|   479|      AC_BC|BC-R205|    Road Bottle Cage|       3|       Road|  2013-07-01|      NULL|
|   477|      AC_BC|WB-H098|Water Bottle - 30...|       2|Other Sales|  2013-07-01|      NULL|
|   483|      AC_BR|RA-H123| Hitch Rack - 4-Bike|      45|Other Sales|  2013-07-01|      NULL|
|   486|      AC_BS|ST-1401|All-Purpose Bike ...|      59|   Mountain|  2013-07-01|      NULL|
|   484|      AC_CL|CL-9009|Bike Wash - Disso...|       3|Other Sales|  2013-07-01|      NULL|
|   485|      AC_FE|FE-6654|Fender Set - Moun...|       8|   Mountain|  2013-07-01|      NULL|
|   215|      AC_HE|HL-U509|Sport-100 Helmet-...| 

In [0]:
silver_crm_sales_details_df.filter((F.col("sls_order_dt") == 0) | (F.length(F.col("sls_order_dt")) < 8)).display()

sls_ord_num,sls_prd_key,sls_cust_id,sls_order_dt,sls_ship_dt,sls_due_dt,sls_sales,sls_quantity,sls_price
SO64338,HL-U509-B,22819,0,20130823,20130828,35,1,35
SO64339,TI-R628,16322,0,20130823,20130828,25,1,25
SO64339,TT-R982,16322,0,20130823,20130828,4,1,4
SO64340,TT-M928,21521,0,20130823,20130828,5,1,5
SO64340,TI-M602,21521,0,20130823,20130828,30,1,30
SO64377,BK-M68S-46,12373,0,20130824,20130829,2320,1,2320
SO64377,GL-H102-S,12373,0,20130824,20130829,24,1,24
SO64379,TI-M267,13008,0,20130824,20130829,25,1,25
SO64381,PK-7098,21995,0,20130824,20130829,2,1,2
SO64623,TI-R092,26791,0,20130827,20130901,21,1,21


In [0]:
silver_crm_sales_details_df = silver_crm_sales_details_df.select(
    "sls_ord_num",
    "sls_prd_key",
    "sls_cust_id",
    F.col("sls_order_dt").cast("string").alias("sls_order_dt"),
    F.col("sls_ship_dt").cast("string").alias("sls_ship_dt"),
    F.col("sls_due_dt").cast("string").alias("sls_due_dt"),
   
    "sls_sales",
    "sls_quantity",
     "sls_price"

)


window = Window.partitionBy("sls_cust_id").orderBy(F.col("sls_cust_id"))


silver_crm_sales_details_df = silver_crm_sales_details_df.select(
    "sls_ord_num",
    "sls_prd_key",
    "sls_cust_id",
    F.when((F.col("sls_order_dt") == "0") | (F.length(F.col("sls_order_dt")) < 8), F.lit(None))
    .otherwise(F.to_date("sls_order_dt", "yyyyMMdd"))
    .alias("sls_order_dt"),
    F.to_date("sls_ship_dt", "yyyyMMdd").alias("sls_ship_dt"),
    F.to_date("sls_due_dt", "yyyyMMdd").alias("sls_due_dt"),
    F.when(F.col("sls_price").isNull(),F.abs(F.col("sls_sales"))/F.abs(F.col("sls_quantity")) )
     .otherwise(F.abs(F.col("sls_price"))).alias("sls_price"),
    F.when(F.col("sls_quantity").isNull(), 0)
     .otherwise(F.abs(F.col("sls_quantity"))).alias("sls_quantity"),
  F.when(F.col("sls_sales").isNull(),F.abs(F.col("sls_price"))*F.abs(F.col("sls_quantity")) )
  .when(F.col("sls_sales")==0,F.abs(F.col("sls_price"))*F.abs(F.col("sls_quantity")) )
     .otherwise(F.abs(F.col("sls_sales"))).alias("sls_sales"),

               

)
silver_crm_sales_details_df.write.mode("overwrite").format("delta")\
.save("/Volumes/etl/medal/silver/crm/crm_sales_details")

In [0]:
w = Window.partitionBy("cst_key").orderBy(F.col("cst_create_date"), F.desc("cst_create_date"))
silver_crm_cust_df\
    .withColumn("row_num", F.row_number().over(w))   \
    .filter((F.col("row_num") > 1) & (F.col("cst_id").isNull()))\
    .display()


    

cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date,row_num


In [0]:
df = silver_erp_CUST_AZ12_df.select(
    F.col("CID").alias("customer_id"),
    F.col("BDATE").alias("birth_date"),
    F.col("GEN").alias("gender")
)
df.show(10)


df= df.na.fill({"gender":"N/A"})

+-------------+----------+------+
|  customer_id|birth_date|gender|
+-------------+----------+------+
|NASAW00011000|1971-10-06|  Male|
|NASAW00011001|1976-05-10|  Male|
|NASAW00011002|1971-02-09|  Male|
|NASAW00011003|1973-08-14|Female|
|NASAW00011004|1979-08-05|Female|
|NASAW00011005|1976-08-01|  Male|
|NASAW00011006|1976-12-02|Female|
|NASAW00011007|1969-11-06|  Male|
|NASAW00011008|1975-07-04|Female|
|NASAW00011009|1969-09-29|  Male|
+-------------+----------+------+
only showing top 10 rows


In [0]:
silver_erp_CUST_AZ12_df=df.withColumn("customer_id", F.substring(F.col("customer_id"), 4, F.length(F.col("customer_id"))))

In [0]:
df= silver_erp_LOC_A101_df.select( F.regexp_replace(F.col("CID"), "-", "").alias("customer_id"),F.col("CNTRY").alias("country"))

In [0]:
silver_erp_LOC_A101_df= df.na.fill({"country":"N/A"})

In [0]:
silver_erp_LOC_A101_df = silver_erp_LOC_A101_df.withColumn(
    "country",   
    F.when(F.trim(F.col("country")) == "", F.lit("n/a"))  
     .otherwise(F.col("country"))                         
)

In [0]:
silver_erp_PX_CAT_G1V2_df.groupby("ID").count().filter(F.col("count") > 1).show()

+---+-----+
| ID|count|
+---+-----+
+---+-----+



In [0]:
 silver_erp_PX_CAT_G1V2_df=silver_erp_PX_CAT_G1V2_df.select(F.col("id"),F.col("CAT").alias("category"),F.col("SUBCAT").alias("sub_category"),F.col("MAINTENANCE").alias("maintenance"))

In [0]:
silver_erp_PX_CAT_G1V2_df.filter(F.col("id") !=F.trim(F.col("id"))).withColumn("spaces",F.length('id')) .display()

id,category,sub_category,maintenance,spaces


In [0]:
silver_erp_CUST_AZ12_df.write.mode("overwrite").format("delta")\
.save("/Volumes/etl/medal/silver/erp/silver_erp_CUST_AZ12")
silver_erp_LOC_A101_df.write.mode("overwrite").format("delta")\
.save("/Volumes/etl/medal/silver/erp/silver_erp_LOC_A101")
silver_erp_PX_CAT_G1V2_df.write.mode("overwrite").format("delta")\
.save("/Volumes/etl/medal/silver/erp/erp_PX_CAT_G1V2")